# Seminararbeit: Konzeption eines minimalen CI-Prozesses für dbt-Projekte

**Kurs:** WWI2023F  
**Dozent:** Andreas Buckenhofer  
**Autor:** Robert Knabe  
**Datum:** 15-02-2026

## Inhaltsverzeichnis

1. [Einleitung und Motivation](#1-einleitung-und-motivation)
   - 1.1 [Problemstellung](#11-problemstellung)
   - 1.2 [Zielsetzung der Arbeit](#12-zielsetzung-der-arbeit)
   - 1.3 [Aufbau der Arbeit](#13-aufbau-der-arbeit)
2. [Grundlagen und fachliche Einordnung](#2-grundlagen-und-fachliche-einordnung)
   - 2.1 [ETL und ELT im Data Engineering](#21-etl-und-elt-im-data-engineering)
   - 2.2 [Rolle von dbt in der Transformationsphase](#22-rolle-von-dbt-in-der-transformationsphase)
   - 2.3 [Continuous Integration im Datenkontext](#23-continuous-integration-im-datenkontext)
   - 2.4 [Observability und Datenqualität](#24-observability-und-datenqualität)
3. [Konzeption eines minimalen CI Prozesses](#3-konzeption-eines-minimalen-ci-prozesses)
   - 3.1 [Rahmenbedingungen und Annahmen](#31-rahmenbedingungen-und-annahmen)
   - 3.2 [Architekturübersicht](#32-architekturübersicht)
   - 3.3 [Designentscheidungen](#33-designentscheidungen)
4. [Technische Umsetzung](#4-technische-umsetzung)
   - 4.1 [Projektstruktur](#41-projektstruktur)
   - 4.2 [Datenmodellierung mit dbt](#42-datenmodellierung-mit-dbt)
   - 4.3 [Definition von Datenqualitätstests](#43-definition-von-datenqualitätstests)
   - 4.4 [CI Pipeline Ablauf](#44-ci-pipeline-ablauf)
   - 4.5 [Beispielhafter CI Lauf](#45-beispielhafter-ci-lauf)
5. [Kritische Würdigung](#5-kritische-würdigung)
   - 5.1 [Stärken des Ansatzes](#51-stärken-des-ansatzes)
   - 5.2 [Limitationen](#52-limitationen)
   - 5.3 [Abgrenzung zu produktiven Setups](#53-abgrenzung-zu-produktiven-setups)
6. [Einsatz von Generativer KI](#6-einsatz-von-generativer-ki)
7. [Fazit und Ausblick](#7-fazit-und-ausblick)
8. [Literaturverzeichnis](#8-literaturverzeichnis)

## 1. Einleitung und Motivation

Link zum GitHub: https://github.com/DerEine117/dbt-ci-pipeline

### 1.1 Problemstellung

In modernen Data Engineering Architekturen werden Daten zunehmend automatisiert verarbeitet und transformiert. Die Qualität dieser Transformationen ist entscheidend für verlässliche Analysen. Fehler in der Transformationslogik können zu verfälschten Ergebnissen und falschen Geschäftsentscheidungen führen.

Oft werden Datenmodelle jedoch manuell getestet oder Änderungen ohne automatisierte Qualitätsprüfungen produktiv gesetzt. Das birgt Risiken für Datenqualität, Reproduzierbarkeit und Nachvollziehbarkeit. Besonders in ELT-Ansätzen, wo Transformationen direkt im Zielsystem stattfinden, ist strukturierte Qualitätssicherung wichtig.

Während Continuous Integration in der Softwareentwicklung etabliert ist, wird sie im Data Engineering häufig vernachlässigt. Werkzeuge wie dbt bieten jedoch strukturierte SQL-Transformationen mit integrierten Tests. Die Frage ist, wie ein minimaler CI-Prozess für dbt-Projekte gestaltet werden kann, um Datenqualität zu verbessern, ohne komplexe Infrastruktur zu erfordern.

### 1.2 Zielsetzung der Arbeit

Diese Arbeit entwirft und implementiert ein minimales CI-Konzept mittels Github Actions für dbt-Projekte. Sie zeigt, wie automatisierte Transformationen und Tests grundlegende Observability im Data Engineering ermöglichen. Der Fokus liegt auf konzeptioneller Einordnung und Designentscheidungen im Rahmen einer Seminararbeit, nicht auf Produktionsreife.

Als technische Basis werden dbt, DuckDB und GitHub Actions verwendet, um einen vollständig lokalen ELT-Prozess mit automatisierter Qualitätssicherung umzusetzen. Die Arbeit überträgt CI-Prinzipien auf den Datenkontext und bewertet kritisch, inwieweit dieser Ansatz zur Verbesserung von Datenqualität und Reproduzierbarkeit beitragen kann.

### 1.3 Aufbau der Arbeit

Nach der Einleitung werden fachliche Grundlagen zu ETL/ELT, dbt, Continuous Integration und Observability vorgestellt. Anschließend folgt das konzeptionelle Design des CI-Prozesses mit Architektur und Designentscheidungen. Die technische Umsetzung beschreibt Projektstruktur, Datenmodelle, Tests und CI-Pipeline. Eine kritische Würdigung diskutiert Stärken und Limitationen.

## 2. Grundlagen und fachliche Einordnung

### 2.1 ETL und ELT im Data Engineering

Im Data Engineering haben sich zwei Paradigmen etabliert: ETL (Extract, Transform, Load) und ELT (Extract, Load, Transform). ETL beschreibt den klassischen Ansatz, bei dem Daten vor dem Laden in das Zielsystem transformiert werden. Dieser Ansatz war verbreitet, als Speicherplatz teuer und Rechenkapazität in Zielsystemen begrenzt war. (Vgl: Vorlesungsskript 03 - Data Managment S. 8 ff.)

Mit modernen Cloud-Warehouses wie Snowflake, BigQuery oder lokalen Lösungen wie DuckDB hat sich ELT durchgesetzt. Hier werden Rohdaten zunächst in ein analytisches Zielsystem geladen und dort transformiert. ELT nutzt die Rechenkapazität moderner Datenbanken und erhält Rohdaten verfügbar. Dies ermöglicht klare Trennung von Integration und Transformation, erleichtert Versionierung und iterative Weiterentwicklung. Transformationen in SQL senken die Einstiegshürde für Analysten.

### 2.2 Rolle von dbt in der Transformationsphase

dbt (data build tool) ist ein Open-Source-Werkzeug für die Transformationsphase in ELT-Prozessen. Es ermöglicht SQL-basierte Transformationen innerhalb analytischer Datenpipelines und konzentriert sich ausschließlich auf Transformation, Dokumentation und Testbarkeit von Datenmodellen.

Ein zentraler Vorteil ist die Modularisierung: Modelle können hierarchisch aufgebaut werden, wobei dbt Abhängigkeiten automatisch auflöst und einen DAG erstellt. Zusätzlich bietet dbt integrierte Tests zur Qualitätssicherung. Tests prüfen auf Spaltenebene etwa fehlende Werte, Eindeutigkeit oder referenzielle Integrität. Diese deklarative Herangehensweise ist mit modernen DevOps-Praktiken vereinbar.

### 2.3 Continuous Integration im Datenkontext

Continuous Integration (CI) bezeichnet die Praxis, Codeänderungen automatisiert zu integrieren und zu testen, um Fehler frühzeitig zu erkennen. Im Data Engineering wird CI seltener eingesetzt als in der Softwareentwicklung, obwohl sich viele Prinzipien übertragen lassen: Transformationsskripte sind Code und sollten versioniert und getestet werden.

Eine CI-Pipeline für dbt führt bei jeder Änderung automatisch Transformationen und Datenqualitätstests durch. Dies erhöht Transparenz und ermöglicht frühzeitige Fehlererkennung. Jede Änderung wird dokumentiert und geprüft, was eine gemeinsame Qualitätsbasis schafft. Allerdings unterscheidet sich CI im Datenkontext: Daten ändern sich zeitlich, Tests sind kontextabhängig und Ausführungsdauer kann höher sein. Dennoch bietet CI einen Mehrwert durch strukturierte, nachvollziehbare Qualitätssicherung.

### 2.4 Observability und Datenqualität

Observability meint die Fähigkeit, Zustand und Qualität von Daten sowie Transformationsprozesse nachvollziehen zu können. Dies umfasst Monitoring, Logging und Alerting. Im Kontext dieser Arbeit wird Observability verstanden als Transparenz über den Zustand der Transformationspipeline durch automatisierte CI-Rückmeldungen und Testergebnisse.

Datenqualität lässt sich anhand von Vollständigkeit, Korrektheit, Konsistenz, Aktualität und Eindeutigkeit bewerten. In dbt können diese Dimensionen durch Tests adressiert werden. Neben funktionalen Datentests ist auch Code-Qualität wichtig. SQL-Linting-Tools wie SQLFluff prüfen Code-Konventionen und Best Practices automatisiert. Dies verbessert Lesbarkeit und Wartbarkeit. Automatisierte Tests liefern kontinuierlich Informationen über Datenmodelle und erzeugen passive Observability. In Kombination mit CI entsteht ein transparenter Workflow zur systematischen Qualitätssicherung.

## 3. Konzeption eines minimalen CI Prozesses

### 3.1 Rahmenbedingungen und Annahmen

Das Konzept basiert auf einem leichtgewichtigen Setup, um Komplexität gering zu halten. Es wird eine dateibasierte Datenbank verwendet. GitHub Actions wurde wegen der Git-Integration und kostenlosen Verfügbarkeit gewählt.

### 3.2 Architekturübersicht

Die Architektur gliedert sich in Git-Repository (Versionskontrolle), CI-Plattform (automatisierte Ausführung) und dateibasierte Datenbank (Ausführungsumgebung). Das Repository enthält Quelldaten als CSV sowie die dbt-Projektstruktur. Bei jeder Änderung wird automatisch ein CI-Lauf ausgelöst, der eine virtuelle Umgebung initialisiert, Abhängigkeiten installiert und alle Transformationen sowie Tests ausführt. Das Ergebnis wird protokolliert und ist im Repository einsehbar, wodurch ein kontinuierlicher Feedback-Loop entsteht.

### 3.3 Designentscheidungen

#### 3.3.1 Einsatz von DuckDB

DuckDB ist eine dateibasierte, spaltenorientierte Datenbank ohne Serverinstallation, die vollständig in Prozessen läuft. Dies vereinfacht die Handhabung in CI-Umgebungen erheblich. DuckDB bietet hohe Performance bei analytischen Abfragen und unterstützt kompatible SQL-Syntax. Es existiert ein offizieller dbt-Adapter, der den Konfigurationsaufwand minimiert. (Vgl. Vorlesungsskripte zu Definitionen und Beispielen)

#### 3.3.2 Einsatz von dbt

dbt bietet eine deklarative, modulare und testbare Herangehensweise. Die Strukturierung in Source-, Staging- und Mart-Schichten entspricht Best Practices und erleichtert Nachvollziehbarkeit. Native Unterstützung für Tests, Dokumentation und Versionierung ist für CI essenziell. Die Integration mit GitHub Actions ist gut dokumentiert.

#### 3.3.3 Einsatz von GitHub Actions

GitHub Actions ist direkt in GitHub integriert und erfordert keine zusätzliche Infrastruktur. Workflows sind deklarativ in YAML definiert und bieten kostenlose Rechenzeit für öffentliche Repositories. Workflows lassen sich flexibel bei verschiedenen Ereignissen auslösen, Logs sind transparent einsehbar.

#### 3.3.4 Staging als View und Mart als Table

Staging-Modelle dienen der initialen Bereinigung und sind als Views materialisiert. Dies spart Speicherplatz und hält die Ausführungsdauer gering. Mart-Modelle sind konsolidierte, aggregierte Datenmodelle und als Tabellen materialisiert, um wiederholte Berechnungen zu vermeiden und schnellere Zugriffszeiten zu ermöglichen.

## 4. Technische Umsetzung

### 4.1 Projektstruktur

Das Projekt folgt dbt-Konventionen. Das Hauptverzeichnis enthält die dbt-Projektkonfiguration und Quelldaten als CSV. Der Ordner models ist in staging und mart unterteilt. Im staging werden Rohdaten normalisiert und typisiert. Jede CSV-Datei wird durch ein Staging-Modell als View repräsentiert, das grundlegende Transformationen wie Spaltenumbenennungen und Datentyp-Konvertierungen durchführt.

Im mart befinden sich konsolidierte Datenmodelle als Tabellen, die geschäftliche Fragestellungen abbilden und Daten aus mehreren Staging-Schichten aggregieren. Zusätzlich sind benutzerdefinierte Tests hinterlegt. Das vollständige Projekt ist als öffentliches GitHub Repository verfügbar, wo alle Code-Dateien, die dbt-Konfiguration sowie die GitHub Actions Workflow-Definitionen eingesehen werden können.

### 4.2 Datenmodellierung mit dbt

Die Modellierung erfolgt in SQL mit Jinja-Template-Syntax. Modelle referenzieren einander über die ref-Funktion, wodurch dbt Abhängigkeiten automatisch auflöst. Jedes Modell enthält Beschreibung und Metadaten zu verwendeten Spalten.

Staging-Modelle transformieren etwa eine Kundentabelle durch Spaltenauswahl, Umbenennung und Datentyp-Konvertierung. Mart-Modelle verknüpfen bereinigte Daten und berechnen aggregierte Kennzahlen. Die Materialisierung als View oder Tabelle wird per Konfiguration festgelegt. Diese Struktur ermöglicht klare Trennung zwischen Datenbereinigung und geschäftlicher Logik.

### 4.3 Definition von Datenqualitätstests

Datenqualität wird durch funktionale Tests auf Datenebene und SQL-Linting auf Code-Ebene sichergestellt. dbt bietet eingebaute Tests wie unique, not_null, accepted_values und relationships, die in YAML-Dateien deklariert werden. Benutzerdefinierte Tests sind als SQL-Abfragen formulierbar und geben bei Fehlschlagen alle Zeilen zurück, die die Bedingung verletzen.

Im Projekt wurden Tests für Eindeutigkeit von Primärschlüsseln, Vollständigkeit kritischer Spalten und referenzielle Integrität definiert. Ergänzend wurde SQLFluff als Linting-Tool integriert. SQLFluff prüft SQL-Code auf Stilrichtlinien und Best Practices. Die Konfiguration erfolgt über eine .sqlfluff-Datei (DuckDB) und Templater (dbt). Durch CI-Integration wird kontinuierlich funktionale Korrektheit und Code-Qualität überwacht.

### 4.4 CI Pipeline Ablauf

Die CI-Pipeline wird durch eine GitHub Actions Workflow-Datei definiert und bei Pushes oder Pull Requests ausgelöst. Der Workflow beginnt mit Repository-Checkout, Python-Installation und Installation der Abhängigkeiten (dbt, DuckDB, SQLFluff). Dann wird dbt deps ausgeführt, gefolgt von dbt seed zum Laden der CSV-Daten in DuckDB.

Mit dbt run werden alle Transformationen gemäß definierter Abhängigkeiten durchgeführt, danach prüft dbt test die funktionalen Tests. Als zusätzlicher Schritt überprüft SQLFluff alle SQL-Modelle auf Stilrichtlinien. Die Ergebnisse werden im Workflow-Log protokolliert. Fehlschläge brechen die Pipeline ab und markieren den Build als fehlgeschlagen, was unmittelbares Feedback über Datenqualität und Code-Qualität gibt. Die vollständige Workflow-Konfiguration ist im GitHub Repository unter .github/workflows einsehbar.

### 4.5 Beispielhafter CI Ablauf

Ein CI-Ablauf beginnt mit dem Klonen des Repositories in die GitHub Actions Umgebung. Python wird installiert, gefolgt von pip install der Abhängigkeiten aus requirements.txt (dbt-core, dbt-duckdb, sqlfluff, sqlfluff-templater-dbt). Die dbt-Befehle werden sequenziell ausgeführt.

Die Ausgabe von dbt run zeigt erfolgreich erstellte Modelle und verarbeitete Zeilen. Bei dbt test werden alle Tests durchlaufen, grüne Markierungen signalisieren Erfolg, rote Markierungen weisen auf Fehler hin. SQLFluff durchsucht anschließend alle SQL-Modelle und prüft gegen definierte Regeln. Die .sqlfluff-Konfiguration legt DuckDB-Dialekt und dbt-Templater fest.

Im Projekt wurden alle Tests und Linting-Prüfungen erfolgreich durchlaufen, was Korrektheit der Transformationslogik, Einhaltung der Qualitätskriterien und Code-Konsistenz bestätigt. Die Ausführungsprotokolle der CI-Läufe können im GitHub Repository unter dem Actions-Tab eingesehen werden, wo jeder Schritt detailliert dokumentiert ist.

## 5. Kritische Würdigung

### 5.1 Stärken des Ansatzes

Der Ansatz ist einfach nachvollziehbar, erfordert keine komplexe Infrastruktur und nutzt kostenfreie Werkzeuge. Die vollständige Versionierung im Git-Repository ermöglicht Nachvollziehbarkeit und Teamarbeit. Durch automatisierte Tests bei jeder Änderung wird kontinuierliche Qualitätssicherung erreicht, Fehler werden frühzeitig erkannt.

Die Integration von SQLFluff erweitert die Qualitätssicherung um Code-Konventionen und Best Practices. Dies führt zu lesbarem, wartbarem und konsistentem Code, was in kollaborativen Projekten wichtig ist. Der Ansatz eignet sich hervorragend als Lernumgebung. Die verwendeten Werkzeuge sind verbreitet und gut dokumentiert, die geringe Komplexität senkt die Einstiegshürde.

### 5.2 Limitationen

Die dateibasierte Datenbank ist nicht für produktive Workloads geeignet, da Skalierbarkeit, Mehrbenutzerfähigkeit und Performance-Optimierung fehlen. Die Ausführungsdauer in CI-Umgebungen ist bei größeren Datenmengen problematisch.

Fehlende Persistenz über CI-Läufe hinweg schließt inkrementelle Transformationen oder historische Analysen aus. In produktiven Setups würde eine zentrale, persistente Datenbank verwendet. Die definierten Tests sind auf strukturelle und syntaktische Prüfungen begrenzt, semantische Probleme werden nicht erkannt. Eine umfassende Data-Quality-Strategie würde zusätzliche Monitoring- und Alerting-Mechanismen erfordern.

### 5.3 Abgrenzung zu produktiven Setups

Produktive Setups nutzen cloud-basierte oder dedizierte Datenbanken wie Snowflake oder BigQuery für bessere Performance und Skalierbarkeit. Orchestrierungs-Tools wie Airflow verwalten komplexe Abhängigkeiten. Umfangreichere Monitoring-Lösungen wie Datadog oder Data-Observability-Plattformen erkennen operative Probleme. Der CI-Prozess würde Code-Reviews, Freigabeprozesse und Deployment-Pipelines umfassen. Verschiedene Umgebungen wären klar getrennt. Der vorliegende Ansatz ist als minimale Variante konzipiert, die Prinzipien veranschaulicht, jedoch nicht alle produktiven Anforderungen erfüllt.

## 6. Einsatz von Generativer KI

Bei der Erstellung dieser Arbeit wurde Copilot als generatives KI System zur Unterstützung eingesetzt.

### 6.1 Verwendete KI-Systeme und Einsatzbereiche


**GitHub Copilot** wurde zur Unterstützung bei der Erstellung von dbt-Modellen und SQL-Abfragen genutzt. Insbesondere bei der Formulierung repetitiver SQL-Statements und der Strukturierung von dbt-Konfigurationsdateien


## 7. Fazit und Ausblick

Diese Arbeit hat gezeigt, wie ein minimaler CI-Prozess für dbt-Projekte konzipiert und umgesetzt werden kann. Durch DuckDB, dbt und GitHub Actions entstand ein leichtgewichtiges Setup, das CI-Prinzipien auf den Datenkontext überträgt. Die automatisierte Ausführung von Transformationen und Tests ermöglicht grundlegende Observability und verbessert Datenqualität.

Die Ergebnisse zeigen, dass bereits mit einfachen Mitteln strukturierte Qualitätssicherung im Data Engineering erreicht werden kann. Der Ansatz ist transparent, nachvollziehbar und gut für Lernzwecke sowie Prototyping geeignet. Die vollständige Implementierung ist als öffentliches GitHub Repository verfügbar, was Nachvollziehbarkeit und Reproduzierbarkeit gewährleistet. Gleichzeitig sind die Grenzen deutlich: Skalierbarkeit und Produktionsreife fehlen.

Zukünftige Erweiterungen könnten Cloud-basierte Data Warehouses integrieren, um realistische produktive Szenarien abzubilden. Auch die Einbindung von Orchestrierungs-Tools wie Airflow oder erweiterte Monitoring-Lösungen wäre denkbar. Die Arbeit leistet einen Beitrag zur Diskussion um Qualitätssicherung und DevOps-Praktiken im Data Engineering. Sie zeigt, dass moderne Werkzeuge strukturierte und nachvollziehbare Datenprozesse ermöglichen, ohne übermäßig komplexe Infrastruktur zu erfordern. Dies ist besonders in frühen Projektphasen oder ressourcenbeschränkten Kontexten wertvoll.

## 8. Literaturverzeichnis

Vorlesungsskripte

https://bix-tech.com/cicd-in-data-engineering-your-essential-guide-to-seamless-data-pipeline-deployment/

https://www.datadoghq.com/blog/understanding-dbt/

https://docs.getdbt.com/docs/deploy/continuous-integration

https://duckdb.org/2025/04/04/dbt-duckdb

https://www.decube.io/post/open-source-data-observability